<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Install-gentle" data-toc-modified-id="Install-gentle-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Install gentle</a></span></li><li><span><a href="#Test-align_transcript" data-toc-modified-id="Test-align_transcript-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test align_transcript</a></span></li></ul></div>

# Install gentle 

1.
git clone https://github.com/lowerquality/gentle.git

2.
in /gentle/install_models.sh:10
change 'https' to 'http'

3.
bash ./install.sh (takes ~30min)   

# Test align_transcript

In [ ]:
from scannertools import audio, transcript_alignment
import scannerpy
import os
from query.models import Video

In [ ]:
video = Video.objects.filter(path__contains='CNNW_20170907_040000_Anderson_Cooper_360')[0]
video2 = Video.objects.filter(path__contains='CNNW_20170605_170000_Wolf')[0]

In [ ]:
aud = audio.AudioSource(video.for_scannertools(), frame_size=60.0)
aud2 = audio.AudioSource(video2.for_scannertools(), frame_size=60.0)

In [ ]:
transcript = [
    '../app/data/CNNW_20170907_040000_Anderson_Cooper_360.cc5.srt',
    '../app/data/CNNW_20170605_170000_Wolf.cc5.srt',
    ]

In [ ]:
db = scannerpy.Database()

In [ ]:
transcript_alignment.align_transcript(db, audio=[aud, aud2], transcript=transcript, cache=False, align_dir='../app/aligned_transcript')